In [ ]:
!wget https://hgdownload.soe.ucsc.edu/goldenPath/hg38/bigZips/hg38.fa.gz

--2020-11-16 05:05:51--  https://hgdownload.soe.ucsc.edu/goldenPath/hg38/bigZips/hg38.fa.gz
Resolving hgdownload.soe.ucsc.edu (hgdownload.soe.ucsc.edu)... 128.114.119.163
Connecting to hgdownload.soe.ucsc.edu (hgdownload.soe.ucsc.edu)|128.114.119.163|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 983659424 (938M) [application/x-gzip]
Saving to: ‘hg38.fa.gz’

hg38.fa.gz          100%[===================>] 938.09M  17.0MB/s    in 91s     

2020-11-16 05:07:23 (10.3 MB/s) - ‘hg38.fa.gz’ saved [983659424/983659424]



In [ ]:
!wget https://hgdownload.soe.ucsc.edu/goldenPath/felCat9/bigZips/felCat9.fa.gz

--2020-11-16 05:33:53--  https://hgdownload.soe.ucsc.edu/goldenPath/felCat9/bigZips/felCat9.fa.gz
Resolving hgdownload.soe.ucsc.edu (hgdownload.soe.ucsc.edu)... 128.114.119.163
Connecting to hgdownload.soe.ucsc.edu (hgdownload.soe.ucsc.edu)|128.114.119.163|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 812014328 (774M) [application/x-gzip]
Saving to: ‘felCat9.fa.gz’

felCat9.fa.gz       100%[===================>] 774.40M  19.1MB/s    in 65s     

2020-11-16 05:35:00 (11.8 MB/s) - ‘felCat9.fa.gz’ saved [812014328/812014328]



In [ ]:
!gunzip hg38.fa.gz

In [ ]:
!gunzip felCat9.fa.gz

In [ ]:
!pip install biopython==1.72

In [ ]:
import Bio

In [ ]:
import numpy as np
from tqdm import tqdm
from Bio import SeqIO, Seq
from Bio.Alphabet import IUPAC

In [ ]:

hg38_genome_file = '/content/hg38.fa'
h38 = SeqIO.index(hg38_genome_file,'fasta', alphabet=IUPAC.unambiguous_dna)
h38_chromosomes = [k for k in h38.keys()]

In [ ]:
dm4R6_genome_file = '/content/felCat9.fa'
dm4R6 = SeqIO.index(dm4R6_genome_file,'fasta', alphabet=IUPAC.unambiguous_dna)
dm4R6_chromosomes = [k for k in dm4R6.keys()]

In [ ]:
h38_chromosomes

['chr1',
 'chr10',
 'chr11',
 'chr11_KI270721v1_random',
 'chr12',
 'chr13',
 'chr14',
 'chr14_GL000009v2_random',
 'chr14_GL000225v1_random',
 'chr14_KI270722v1_random',
 'chr14_GL000194v1_random',
 'chr14_KI270723v1_random',
 'chr14_KI270724v1_random',
 'chr14_KI270725v1_random',
 'chr14_KI270726v1_random',
 'chr15',
 'chr15_KI270727v1_random',
 'chr16',
 'chr16_KI270728v1_random',
 'chr17',
 'chr17_GL000205v2_random',
 'chr17_KI270729v1_random',
 'chr17_KI270730v1_random',
 'chr18',
 'chr19',
 'chr1_KI270706v1_random',
 'chr1_KI270707v1_random',
 'chr1_KI270708v1_random',
 'chr1_KI270709v1_random',
 'chr1_KI270710v1_random',
 'chr1_KI270711v1_random',
 'chr1_KI270712v1_random',
 'chr1_KI270713v1_random',
 'chr1_KI270714v1_random',
 'chr2',
 'chr20',
 'chr21',
 'chr22',
 'chr22_KI270731v1_random',
 'chr22_KI270732v1_random',
 'chr22_KI270733v1_random',
 'chr22_KI270734v1_random',
 'chr22_KI270735v1_random',
 'chr22_KI270736v1_random',
 'chr22_KI270737v1_random',
 'chr22_KI270738v1_ra

In [ ]:
def get_random_sequence(genome, chromosomes, l, N) :
    """
    Get a random sequence of nucleotides from the genome. It will
    be a sequence of N words of l letters.
    """
    
    # first, pick a random chromosome
    chromosome = genome[np.random.choice(chromosomes)]
    
    # then pick a random spot in the chromosome
    found = False
    chromosome_size = len(chromosome.seq)
    while not found :
        start = np.random.choice(chromosome_size)
        myseq = chromosome.seq[start:start+l*N]
        
        # here we take care that there are no 'N's in the sequence,
        # and that the length of the sequence is good
        if 'N' not in myseq and len(myseq)==l*N :
            found = True
    s = str(myseq).upper()
    return [s[0+i:l+i] for i in range(0, l*N, l)]

In [ ]:

def sequence_encoder(sequence, mapping) :
    l = len(sequence)
    return np.sum([4**(l-i-1)*mapping[sequence[i]] for i in range(l)])

In [ ]:
l = 4
N = 10
s = get_random_sequence(dm4R6, dm4R6_chromosomes, l, N)

In [ ]:

mapping = {
    'A' : 0,
    'T' : 1,
    'C' : 2,
    'G' : 3
}

In [ ]:
sequence_encoder('GCC', mapping)


58

In [ ]:
ndata = 100000
l = 7
N = 80
human_fname = '/content/human.dataset'
droso_fname = '/content/cat.dataset'
fh = open(human_fname, 'w')
fd = open(droso_fname, 'w')
for n in tqdm(range(ndata)) :
    human_seq = get_random_sequence(h38, h38_chromosomes, l, N)
    droso_seq = get_random_sequence(dm4R6, dm4R6_chromosomes, l, N)
    for i in range(N) :
        fh.write('%d\t'%(sequence_encoder(human_seq[i], mapping)))
        fd.write('%d\t'%(sequence_encoder(droso_seq[i], mapping)))
    fh.write('\n')
    fd.write('\n')
fh.close()
fd.close()

 11%|█         | 11047/100000 [35:39<17:05:46,  1.45it/s]

KeyError: ignored

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding
from tensorflow.keras.layers import LSTM
from tensorflow.keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D

In [ ]:
!grep -v '^>' /content/hg38.fa >/content/hg38.txt

In [ ]:
f = open("/content/hg38.txt", "r")
print(f.readline())

NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN



In [ ]:
for x in f:
  print(x)

Streaming output truncated to the last 5000 lines.

...


In [ ]:
f = open("/content/hg38.txt", "r")

In [ ]:
li=[]
for x in f:
  li.append(x)

In [ ]:
li

['...\n',
 ...]

In [ ]:
list3=[]
def convertOpposite(str): 
    ln = len(str) 
  
    # Conversion according to ASCII values 
    for i in range(ln):
  
      if str[i] >= 'A' and str[i] <= 'Z': 
  
            # Convert lowercase to uppercase 
            str[i] = chr(ord(str[i]) + 32) 
  
# Driver code 
if __name__ == "__main__": 
    #f = open("/content/hg38.txt", "r")
    #str=list(f)
    for i in range(len(list2)):
      for x in list2:
          lower=convertOpposite(x)
          list3.append(lower)
      #= list2[i]
  
      #convertOpposite(str1) 
      #str1 = ''.join(str1) 
      #print(str1) 
  

In [ ]:
f = open("/content/hg38.txt", "r")
str1=list(f)

In [ ]:
human="""'...\n',
"""
       


In [ ]:
for s in "\n,"   :
    human = human.replace(human, "")

In [ ]:
print(human)

In [ ]:
lower=convertOpposite(list2)

TypeError: ignored

In [ ]:
human="""
"""

In [ ]:
list2=[ '...\n',
 ]

In [ ]:
from numpy.random import choice
import numpy as np 
import time
import tracemalloc
import gzip 


In [ ]:
def consume_cpu_ram(n): return np.ones((n, n))
def consume_cpu_ram_128mb(): return consume_cpu_ram(2**12)

In [ ]:
z = [consume_cpu_ram_128mb() for i in range(8)] # 1GB
del z
tracemalloc.start()
a1 = consume_cpu_ram_128mb()
a2 = consume_cpu_ram_128mb()
del a1

In [ ]:
start = time.time()
sequence = open('/content/hg38.fa','r').read()

base_tallies = []
base_tallies.append(sequence.count('a'))
base_tallies.append(sequence.count('g'))    
base_tallies.append(sequence.count('c')) 
base_tallies.append(sequence.count('t'))

base_tallies = np.array(base_tallies).astype(float)

sequence_length = float(len(sequence))

probabilities = base_tallies /sequence_length

print(f'Sum of probs: {np.sum(probabilities)}')
print(f'Probs       : {probabilities}')
probabilities = probabilities/np.sum(probabilities)

print(f'Prob A: {probabilities[0]:0.3f}')
print(f'Prob G: {probabilities[1]:0.3f}')
print(f'Prob C: {probabilities[2]:0.3f}')
print(f'Prob T: {probabilities[3]:0.3f}')

bases = ['a','g','c','t']

Sum of probs: 0.485303544210114
Probs       : [0.14169647 0.1010091  0.1002781  0.14231988]
Prob A: 0.292
Prob G: 0.208
Prob C: 0.207
Prob T: 0.293


In [ ]:
output_sequence_zero = []
for i in range(100): 
    output_sequence_zero.append(choice(bases,p=probabilities))

In [ ]:
log2_probability_zero = 0

for base in sequence:
    base_index = bases.index(base)
    log2_probability_zero += np.log2(probabilities[base_index])

print(f'Log2Prob for Zeroth Order: {log2_probability_zero}')

In [ ]:
transition_probabilities = np.zeros(shape=(4,4))

In [ ]:
base_patterns = np.array([
['aa','at','ac','ag'],
['ta','tt','tc','tg'],
['ca','ct','cc','cg'],
['ga','gt','gc','gg']
])

In [ ]:
print('First Order Count')
for i in range(len(sequence) - 1):
    if (sequence[i] + sequence[i+1]) in base_patterns:
        x = np.where(base_patterns==(sequence[i]+sequence[i+1]))[0][0]
        y = np.where(base_patterns==(sequence[i]+sequence[i+1]))[1][0]
        transition_probabilities[x][y] += 1
print(transition_probabilities)
for row in transition_probabilities:
    row /= np.sum(row)

print('Probability 1st')
print(transition_probabilities)
print(np.sum(transition_probabilities))
sequence = sequence.casefold()

print(f'Length of sequence:{len(sequence)}')

First Order Count


In [ ]:
string='attaaaggtttataccttcccaggtaacaaaccaaccaactttcgatctcttgtagatct'

In [ ]:
print(len(string))

60


In [ ]:
countA=0
countT=0
countG=0
countC=0
for i in range(len(char1)):
  if char1[i] =='a':
    countA += 1
  elif char1[i] =='t':
    countT += 1
  elif char1[i] =='g':
    countG += 1
  else : 
    countC += 1      
print("number of a's in string = ",countA)   
print("number of t's in string = ",countT)   
print("number of g's in string = ",countG)   
print("number of c's in string = ",countC)    

number of a's in string =  19
number of t's in string =  19
number of g's in string =  7
number of c's in string =  15


In [ ]:
char1='attaaaggtttataccttcccaggtaacaaaccaaccaactttcgatctcttgtagatct'

In [ ]:
len(char1)

60

In [ ]:
covid_dna = """
1 attaaaggtt tataccttcc caggtaacaa accaaccaac tttcgatctc ttgtagatct    
61 gttctctaaa cgaactttaa aatctgtgtg gctgtcactc ggctgcatgc ttagtgcact
121 cacgcagtat aattaataac taattactgt cgttgacagg acacgagtaa ctcgtctatc
181 ttctgcaggc tgcttacggt ttcgtccgtg ttgcagccga tcatcagcac atctaggttt
241 cgtccgggtg tgaccgaaag gtaagatgga gagccttgtc cctggtttca acgagaaaac
301 acacgtccaa ctcagtttgc ctgttttaca ggttcgcgac gtgctcgtac gtggctttgg
361 agactccgtg gaggaggtct tatcagaggc acgtcaacat cttaaagatg gcacttgtgg
421 cttagtagaa gttgaaaaag gcgttttgcc tcaacttgaa cagccctatg tgttcatcaa
481 acgttcggat gctcgaactg cacctcatgg tcatgttatg gttgagctgg tagcagaact
541 cgaaggcatt cagtacggtc gtagtggtga gacacttggt gtccttgtcc ctcatgtggg
601 cgaaatacca gtggcttacc gcaaggttct tcttcgtaag aacggtaata aaggagctgg


In [ ]:
base_patterns = np.array([
['aa','at','ac','ag'],
['ta','tt','tc','tg'],
['ca','ct','cc','cg'],
['ga','gt','gc','gg']
])

In [ ]:
count_A=0
count_T=0
count_G=0
count_C=0
count_TA=0
count_AT=0

for i in range(len(char1)):
  if char1[i] =='a':
    count_A += 1
  elif char1[i] =='t':
    count_T += 1
  elif char1[i] =='g':
    count_G += 1
  elif char1[i]=='c' : 
    count_C += 1
  elif char1[i]=='a' and char1[i+1]=='t': 
    count_AT += 1
  elif char1[i]+char1[i+1]=='ta': 
    count_TA += 1         
print("number of a's in string = ",count_A)   
print("number of t's in string = ",count_T)   
print("number of g's in string = ",count_G)   
print("number of c's in string = ",count_C) 
print("number of at's in string = ",count_AT) 
print("number of ta's in string = ",count_TA) 

number of a's in string =  19
number of t's in string =  19
number of g's in string =  7
number of c's in string =  15
number of at's in string =  0
number of ta's in string =  0


In [ ]:
lis1=['aa','at','ag','ac','tt','ta','tg','tc','gg','ga','gt','gc','cc','ca','ct','cg']
lis2=


In [ ]:
lis1[0]

'aa'

In [ ]:
lis1[8]

'gg'

In [ ]:
count_AA=0
for i in range(len(char1)):
  
  conca=char1[i]+char1[i+1]
  print(conca)
  #if lis1[0] == conca:
   # count_AA += 1
#print(count_AA)  

at
tt
ta
aa
aa
ag
gg
gt
tt
tt
ta
at
ta
ac
cc
ct
tt
tc
cc
cc
ca
ag
gg
gt
ta
aa
ac
ca
aa
aa
ac
cc
ca
aa
ac
cc
ca
aa
ac
ct
tt
tt
tc
cg
ga
at
tc
ct
tc
ct
tt
tg
gt
ta
ag
ga
at
tc
ct


IndexError: ignored

In [ ]:
for i in range(len(char1)):
  r=i+1%2
  j=r+1
  print(j)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60


In [ ]:
|